In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

from scipy import stats

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [8]:
gc.collect()

226

In [9]:
path = 'D:/data/'
data = pd.read_feather(path+'jet_complex_data/data_jet_level.feather')
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672504 entries, 0 to 1672503
Data columns (total 144 columns):
jet_id                                1672504 non-null object
number_of_particles_in_this_jet       1672504 non-null int8
jet_px                                1672504 non-null float32
jet_py                                1672504 non-null float32
jet_pz                                1672504 non-null float32
jet_energy                            1672504 non-null float32
jet_mass                              1672504 non-null float32
event_id                              1672504 non-null object
label                                 1134555 non-null float32
jet_energy/num                        1672504 non-null float32
jet_mass/num                          1672504 non-null float32
jet_px/num                            1672504 non-null float32
jet_py/num                            1672504 non-null float32
jet_pz/num                            1672504 non-null float32
jet_|p|  

In [10]:
data

,jet_id,number_of_particles_in_this_jet,jet_px,jet_py,jet_pz,jet_energy,jet_mass,event_id,label,jet_energy/num,jet_mass/num,jet_px/num,jet_py/num,jet_pz/num,jet_|p|,jet_pt,jet_phi,jet_eta,jet_theta,jet_phi_cos_abs,jet_eta_sinh_abs,jet_theta_cos_abs,jet_energy_differ,jet_energy_differ/num,jet_mass/pt,jet_id_particle_px_mean,jet_id_particle_px_max,jet_id_particle_px_min,jet_id_particle_px_sum,jet_id_particle_px_std,jet_id_particle_py_mean,jet_id_particle_py_max,jet_id_particle_py_min,jet_id_particle_py_sum,jet_id_particle_py_std,jet_id_particle_pz_mean,jet_id_particle_pz_max,jet_id_particle_pz_min,jet_id_particle_pz_sum,jet_id_particle_pz_std,jet_id_particle_energy_mean,jet_id_particle_energy_max,jet_id_particle_energy_min,jet_id_particle_energy_sum,jet_id_particle_energy_std,jet_id_particle_mass_mean,jet_id_particle_mass_max,jet_id_particle_mass_min,jet_id_particle_mass_sum,jet_id_particle_mass_std,...,jet_id_particle_eta_sinh_abs_mean,jet_id_particle_eta_sinh_abs_max,jet_id_particle_eta_sinh_abs_min,jet_id_particle_eta_sinh_abs_sum,jet_id_particle_eta_sinh_abs_std,jet_id_particle_theta_cos_abs_mean,jet_id_particle_theta_cos_abs_max,jet_id_particle_theta_cos_abs_min,jet_id_particle_theta_cos_abs_sum,jet_id_particle_theta_cos_abs_std,jet_id_particle_energy_differ_mean,jet_id_particle_energy_differ_max,jet_id_particle_energy_differ_min,jet_id_particle_energy_differ_sum,jet_id_particle_energy_differ_std,jet_id_particle_mass/pt_mean,jet_id_particle_mass/pt_max,jet_id_particle_mass/pt_min,jet_id_particle_mass/pt_sum,jet_id_particle_mass/pt_std,jet_id_particle_phi_rel_mean,jet_id_particle_phi_rel_max,jet_id_particle_phi_rel_min,jet_id_particle_phi_rel_sum,jet_id_particle_phi_rel_std,jet_id_particle_eta_rel_mean,jet_id_particle_eta_rel_max,jet_id_particle_eta_rel_min,jet_id_particle_eta_rel_sum,jet_id_particle_eta_rel_std,jet_id_particle_theta_rel_mean,jet_id_particle_theta_rel_max,jet_id_particle_theta_rel_min,jet_id_particle_theta_rel_sum,jet_id_particle_theta_rel_std,jet_id_particle_pt_rel_mean,jet_id_particle_pt_rel_max,jet_id_particle_pt_rel_min,jet_id_particle_pt_rel_sum,jet_id_particle_pt_rel_std,jet_id_particle_energy_rel_mean,jet_id_particle_energy_rel_max,jet_id_particle_energy_rel_min,jet_id_particle_energy_rel_sum,jet_id_particle_energy_rel_std,jet_id_particle_R_mean,jet_id_particle_R_max,jet_id_particle_R_min,jet_id_particle_R_sum,jet_id_particle_R_std
0,17a0a1f19a7f43558cb9cca517b6e198,21,-288.511993,0.302560,1.839620,289.066010,17.781099,410e86bc0457482d9d6f81cd2d95ccf0,1.0,13.765048,0.846719,13.738667,0.014408,0.087601,288.518036,288.512146,3.140557,0.006376,1.564420,0.999999,0.006376,0.006376,0.547974,0.026094,0.061630,-13.738700,-0.092363,-146.042999,-288.512695,32.159996,0.014408,0.785048,-0.610681,0.302561,0.319108,0.087601,0.969680,-0.570297,1.839623,0.302749,13.772425,146.046005,0.095254,289.220917,32.148827,0.114773,0.93957,0.0,2.410240,0.222063,...,0.088200,0.321242,0.001227,1.852198,0.099020,0.086339,0.305848,0.001227,1.813126,0.095892,0.010053,0.136289,-0.000626,0.211117,0.030086,0.056251,0.466360,0.0,1.181262,0.120566,-0.104815,-0.003304,-0.323539,-2.201106,0.105882,0.020554,0.309583,-0.227097,0.431627,0.129940,-0.020322,0.225326,-0.304453,-0.426764,0.128766,0.047662,0.506201,0.000322,1.000896,0.111455,0.047645,0.505234,0.000330,1.000536,0.111216,0.142401,0.355421,0.004471,2.990414,0.137049
1,36d1005f56d343d5a09513cb7a4ead61,26,-158.447998,2.132650,-16.283400,159.718002,11.584800,6f7bf9104f8c4a619eedd129d5ce3dcb,5.0,6.143000,0.445569,6.094154,0.082025,0.626285,159.296783,158.462357,3.128132,-0.102579,1.673196,0.999909,0.102759,0.102221,0.421219,0.016201,0.073108,-6.094159,-0.261038,-26.073500,-158.448120,6.783796,0.082025,0.885232,-0.624319,2.132648,0.329125,-0.626286,0.061388,-3.275600,-16.283442,0.817435,6.145245,26.237400,0.263444,159.776352,6.829412,0.136642,0.93957,0.0,3.552680,0.245883,...,0.101645,0.208076,0.013674,2.642768,0.048130,0.100793,0.203713,0.013672,2.620626,0.047261,0.003404,0.042799,-0.000

In [11]:
data.loc[data['event_id']=='4ee694b125c944c1a4560a20b79ccbe3']

,jet_id,number_of_particles_in_this_jet,jet_px,jet_py,jet_pz,jet_energy,jet_mass,event_id,label,jet_energy/num,jet_mass/num,jet_px/num,jet_py/num,jet_pz/num,jet_|p|,jet_pt,jet_phi,jet_eta,jet_theta,jet_phi_cos_abs,jet_eta_sinh_abs,jet_theta_cos_abs,jet_energy_differ,jet_energy_differ/num,jet_mass/pt,jet_id_particle_px_mean,jet_id_particle_px_max,jet_id_particle_px_min,jet_id_particle_px_sum,jet_id_particle_px_std,jet_id_particle_py_mean,jet_id_particle_py_max,jet_id_particle_py_min,jet_id_particle_py_sum,jet_id_particle_py_std,jet_id_particle_pz_mean,jet_id_particle_pz_max,jet_id_particle_pz_min,jet_id_particle_pz_sum,jet_id_particle_pz_std,jet_id_particle_energy_mean,jet_id_particle_energy_max,jet_id_particle_energy_min,jet_id_particle_energy_sum,jet_id_particle_energy_std,jet_id_particle_mass_mean,jet_id_particle_mass_max,jet_id_particle_mass_min,jet_id_particle_mass_sum,jet_id_particle_mass_std,...,jet_id_particle_eta_sinh_abs_mean,jet_id_particle_eta_sinh_abs_max,jet_id_particle_eta_sinh_abs_min,jet_id_particle_eta_sinh_abs_sum,jet_id_particle_eta_sinh_abs_std,jet_id_particle_theta_cos_abs_mean,jet_id_particle_theta_cos_abs_max,jet_id_particle_theta_cos_abs_min,jet_id_particle_theta_cos_abs_sum,jet_id_particle_theta_cos_abs_std,jet_id_particle_energy_differ_mean,jet_id_particle_energy_differ_max,jet_id_particle_energy_differ_min,jet_id_particle_energy_differ_sum,jet_id_particle_energy_differ_std,jet_id_particle_mass/pt_mean,jet_id_particle_mass/pt_max,jet_id_particle_mass/pt_min,jet_id_particle_mass/pt_sum,jet_id_particle_mass/pt_std,jet_id_particle_phi_rel_mean,jet_id_particle_phi_rel_max,jet_id_particle_phi_rel_min,jet_id_particle_phi_rel_sum,jet_id_particle_phi_rel_std,jet_id_particle_eta_rel_mean,jet_id_particle_eta_rel_max,jet_id_particle_eta_rel_min,jet_id_particle_eta_rel_sum,jet_id_particle_eta_rel_std,jet_id_particle_theta_rel_mean,jet_id_particle_theta_rel_max,jet_id_particle_theta_rel_min,jet_id_particle_theta_rel_sum,jet_id_particle_theta_rel_std,jet_id_particle_pt_rel_mean,jet_id_particle_pt_rel_max,jet_id_particle_pt_rel_min,jet_id_particle_pt_rel_sum,jet_id_particle_pt_rel_std,jet_id_particle_energy_rel_mean,jet_id_particle_energy_rel_max,jet_id_particle_energy_rel_min,jet_id_particle_energy_rel_sum,jet_id_particle_energy_rel_std,jet_id_particle_R_mean,jet_id_particle_R_max,jet_id_particle_R_min,jet_id_particle_R_sum,jet_id_particle_R_std
2,a82071a653304481979b5dfb8eb0da47,27,-104.142998,-37.26720,-20.658501,113.320000,13.42170,4ee694b125c944c1a4560a20b79ccbe3,4.0,4.197037,0.497100,3.857148,1.380267,0.765130,112.522804,110.610161,2.797945,-0.185699,1.755438,0.941532,0.186769,0.183594,0.797195,0.029526,0.121342,-3.857141,-0.075788,-23.148500,-104.142815,5.920946,-1.380266,-0.014294,-8.119850,-37.267170,2.102180,-0.765130,0.185358,-5.108020,-20.658518,1.363466,4.199011,25.05740,0.077390,113.373306,6.411531,0.117459,0.93827,0.0,3.17139,0.177539,...,0.170439,0.410238,0.019022,4.601863,0.102618,0.165524,0.379542,0.019018,4.469160,0.096396,0.008213,0.054320,-7.247925e-05,0.221764,0.013534,0.098375,0.649158,0.0,2.656124,0.154053,0.025301,0.301587,-0.175502,0.683119,0.130078,0.044398,0.329143,-0.213825,1.198756,0.137588,-0.044921,0.204660,-0.327595,-1.212854,0.135677,0.037114,0.221782,0.000697,1.002065,0.056752,0.037054,0.221121,0.000683,1.000470,0.056579,0.160529,0.358548,0.019035,4.334296,0.108635
866832,b04cc67721df48c8adb9f3e8c8bd5b0f,24,180.710007,-2.87614,3.852040,181.447998,15.63180,4ee694b125c944c1a4560a20b79ccbe3,4.0,7.560333,0.651325,7.529583,0.119839,0.160502,180.773941,180.732895,0.015916,0.021312,1.549486,0.999873,0.021313,0.021309,0.674057,0.028086,0.086491,7.529569,24.932600,0.104397,180.709656,8.558002,-0.119839,1.020560,-1.328450,-2.876138,0.643826,0.160502,0.899003,-0.308666,3.852040,0.304058,7.568460,24.96620,0.105908,181.643036,8.559230,0.130582,0.93957,0.0,3.13397,0.257852,...,0.067979,0.190168,0.006020,1.631501,0.049064,0.067553,0.186820,0.006020,1.621265,0.048404,0.008887,0.104380,-5.364418e-0

In [12]:
train = data.loc[data['label'].isna()==False]
test = data.loc[data['label'].isna()==True]

In [13]:
label_map = dict(zip(np.sort(train['label'].unique()), range(len(train['label'].unique()))))
label_map_reverse = dict(zip(range(len(train['label'].unique())), np.sort(train['label'].unique())))

In [14]:
train['label'] = train['label'].map(label_map)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
drop_feat = ['jet_id_particle_px_mean',
 'jet_id_particle_px_max',
 'jet_id_particle_px_min',
 'jet_id_particle_px_sum',
 'jet_id_particle_px_std',
 'jet_id_particle_py_mean',
 'jet_id_particle_py_max',
 'jet_id_particle_py_min',
 'jet_id_particle_py_sum',
 'jet_id_particle_py_std',
 'jet_id_particle_pz_mean',
 'jet_id_particle_pz_max',
 'jet_id_particle_pz_min',
 'jet_id_particle_pz_sum',
 'jet_id_particle_pz_std',
 'jet_id_particle_energy_mean',
 'jet_id_particle_energy_max',
 'jet_id_particle_energy_min',
 'jet_id_particle_energy_sum',
 'jet_id_particle_energy_std',
 'jet_id_particle_mass_mean',
 'jet_id_particle_mass_max',
 'jet_id_particle_mass_min',
 'jet_id_particle_mass_sum',
 'jet_id_particle_mass_std',
 'jet_id_particle_category_-2212_sum',
 'jet_id_particle_category_-2112_sum',
 'jet_id_particle_category_-321_sum',
 'jet_id_particle_category_-211_sum',
 'jet_id_particle_category_-13_sum',
 'jet_id_particle_category_-11_sum',
 'jet_id_particle_category_11_sum',
 'jet_id_particle_category_13_sum',
 'jet_id_particle_category_22_sum',
 'jet_id_particle_category_130_sum',
 'jet_id_particle_category_211_sum',
 'jet_id_particle_category_321_sum',
 'jet_id_particle_category_2112_sum',
 'jet_id_particle_category_2212_sum',
 'jet_id_particle_|p|_mean',
 'jet_id_particle_|p|_max',
 'jet_id_particle_|p|_min',
 'jet_id_particle_|p|_sum',
 'jet_id_particle_|p|_std',
 'jet_id_particle_pt_mean',
 'jet_id_particle_pt_max',
 'jet_id_particle_pt_min',
 'jet_id_particle_pt_sum',
 'jet_id_particle_pt_std',
 'jet_id_particle_phi_mean',
 'jet_id_particle_phi_max',
 'jet_id_particle_phi_min',
 'jet_id_particle_phi_sum',
 'jet_id_particle_phi_std',
 'jet_id_particle_eta_mean',
 'jet_id_particle_eta_max',
 'jet_id_particle_eta_min',
 'jet_id_particle_eta_sum',
 'jet_id_particle_eta_std',
 'jet_id_particle_theta_mean',
 'jet_id_particle_theta_max',
 'jet_id_particle_theta_min',
 'jet_id_particle_theta_sum',
 'jet_id_particle_theta_std',
 'jet_id_particle_phi_cos_abs_mean',
 'jet_id_particle_phi_cos_abs_max',
 'jet_id_particle_phi_cos_abs_min',
 'jet_id_particle_phi_cos_abs_sum',
 'jet_id_particle_phi_cos_abs_std',
 'jet_id_particle_eta_sinh_abs_mean',
 'jet_id_particle_eta_sinh_abs_max',
 'jet_id_particle_eta_sinh_abs_min',
 'jet_id_particle_eta_sinh_abs_sum',
 'jet_id_particle_eta_sinh_abs_std',
 'jet_id_particle_theta_cos_abs_mean',
 'jet_id_particle_theta_cos_abs_max',
 'jet_id_particle_theta_cos_abs_min',
 'jet_id_particle_theta_cos_abs_sum',
 'jet_id_particle_theta_cos_abs_std',
 'jet_id_particle_energy_differ_mean',
 'jet_id_particle_energy_differ_max',
 'jet_id_particle_energy_differ_min',
 'jet_id_particle_energy_differ_sum',
 'jet_id_particle_energy_differ_std',
 'jet_id_particle_mass/pt_mean',
 'jet_id_particle_mass/pt_max',
 'jet_id_particle_mass/pt_min',
 'jet_id_particle_mass/pt_sum',
 'jet_id_particle_mass/pt_std',
 'jet_id_particle_phi_rel_mean',
 'jet_id_particle_phi_rel_max',
 'jet_id_particle_phi_rel_min',
 'jet_id_particle_phi_rel_sum',
 'jet_id_particle_phi_rel_std',
 'jet_id_particle_eta_rel_mean',
 'jet_id_particle_eta_rel_max',
 'jet_id_particle_eta_rel_min',
 'jet_id_particle_eta_rel_sum',
 'jet_id_particle_eta_rel_std',
 'jet_id_particle_theta_rel_mean',
 'jet_id_particle_theta_rel_max',
 'jet_id_particle_theta_rel_min',
 'jet_id_particle_theta_rel_sum',
 'jet_id_particle_theta_rel_std',
 'jet_id_particle_pt_rel_mean',
 'jet_id_particle_pt_rel_max',
 'jet_id_particle_pt_rel_min',
 'jet_id_particle_pt_rel_sum',
 'jet_id_particle_pt_rel_std',
 'jet_id_particle_energy_rel_mean',
 'jet_id_particle_energy_rel_max',
 'jet_id_particle_energy_rel_min',
 'jet_id_particle_energy_rel_sum',
 'jet_id_particle_energy_rel_std',
 'jet_id_particle_R_mean',
 'jet_id_particle_R_max',
 'jet_id_particle_R_min',
 'jet_id_particle_R_sum',
 'jet_id_particle_R_std']

In [16]:
used_feat = [f for f in test.columns if f not in ['event_id', 'label', 'jet_id'] + list(drop_feat)]
len(used_feat)
used_feat

22

['number_of_particles_in_this_jet',
 'jet_px',
 'jet_py',
 'jet_pz',
 'jet_energy',
 'jet_mass',
 'jet_energy/num',
 'jet_mass/num',
 'jet_px/num',
 'jet_py/num',
 'jet_pz/num',
 'jet_|p|',
 'jet_pt',
 'jet_phi',
 'jet_eta',
 'jet_theta',
 'jet_phi_cos_abs',
 'jet_eta_sinh_abs',
 'jet_theta_cos_abs',
 'jet_energy_differ',
 'jet_energy_differ/num',
 'jet_mass/pt']

In [17]:
train_x = train[used_feat].reset_index(drop=True)
train_y = train['label'].reset_index(drop=True)
test_x = test[used_feat].reset_index(drop=True)

In [18]:
def multi_auc(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score.reshape((-1,4)), axis=1)))
    return 'multi_auc', score, True

def multi_auc_oof(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score, axis=1)))
    return score

In [19]:
preds = np.zeros((test_x.shape[0], 4))
oof_lgb = np.zeros((train_x.shape[0], 4))
scores = []

imp = pd.DataFrame()
imp['feat'] = used_feat

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=527)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*'*120)
    print('Fold %d' % (index+1))
    print('*'*120)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]
    eval_set = [(X_valid, y_valid)]
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=2**6-1, max_depth=6, reg_alpha=0, reg_lambda=1.9, 
                                    max_bin=64, n_estimators=10000, objective='multiclass', num_class=4, 
                                    bagging_fraction=0.8, bagging_freq=5, min_child_samples=80, 
                                    feature_fraction=0.8, learning_rate=0.05, random_state=527, n_jobs=4)
    
    model.fit(X_train, y_train, eval_set=eval_set,verbose=300, early_stopping_rounds=500,
#                   eval_metric=multi_auc, 
#                   categorical_feature=catlist
                 )
    
    imp['score%d' % (index+1)] = model.feature_importances_
    
    score = model.best_score_['valid_0']['multi_logloss']
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, model.best_iteration_, score,np.mean(scores)))
    proba =  model.predict_proba(X_valid)
    print('auc: ',multi_auc_oof(y_valid, proba))
    print('\n\n')
    oof_lgb[va_idx] = proba
    preds += model.predict_proba(test_x)
    break

************************************************************************************************************************
Fold 1
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 1.24835
[600]	valid_0's multi_logloss: 1.24872
Early stopping, best iteration is:
[318]	valid_0's multi_logloss: 1.24832


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 1 round 318 : score: 1.248319 | mean score 1.248319
auc:  0.5952119167273962





In [20]:
#data_new fold1
# fold 1 round 2757 : score: 0.817910 | mean score 0.817910
# auc:  0.771650968641868

print(scores)
print('logloss:',np.mean(scores))
print('auc:', multi_auc_oof(train_y, oof_lgb))
print('acc:', accuracy_score(train_y, np.argmax(oof_lgb, axis=1)))
# logloss: 0.8394470890181213 auc: 0.7639691112376329 acc: 0.6468655840291672  featv0 0.75723 $
# logloss: 0.8386205747554791 auc: 0.7641546488063027 acc: 0.6471483155930496 featv1 0.75757 $
# logloss: 0.8385694282652787 auc: 0.7644872386334696 acc: 0.6476437690002342 527 527
# logloss: 0.83799263525929 auc: 0.7643224628185179 acc: 0.6474014276597635 featv2 527 527 0.75785 $
# logloss: 0.8230686055330182 auc: 0.7691376036021507 acc: 0.6545720386561364 featv3 0.76301 $
# logloss: 0.8153214789014147 auc: 0.772159018172669 acc: 0.6590661241810882 data_ 0.76654 $
# logloss: 0.8151705994830186 auc: 0.7722130382430792 acc: 0.6591495973094725 data_:1000 0.76649
# logloss: 0.8154789314371726 auc: 0.7720832773246235 acc: 0.6589530315555352 data_:1200 0.76588
# logloss: 0.8152194259449506 auc: 0.7722792166028103 acc: 0.6592411484825393 data_:800  0.76618
# logloss: 0.8157724809172372 auc: 0.7721843858406869 acc: 0.6591038217229392 data_:1400  0.76606
# logloss: 0.815500754326537 auc: 0.7718635245268304 acc: 0.6586245244051193 data_:1500  0.76559
# logloss: 0.8154257388311787 auc: 0.7719421140288143 acc: 0.6587456950753547 data_:1540 

# logloss: 0.8153689546673759 auc: 0.77196372533444 acc: 0.6587806999356449 data_new X 0.76571
# logloss: 0.8151418696703783 auc: 0.772302807868475 acc: 0.6592842313875119 data_new 0.76631

[1.2483191285951007]
logloss: 1.2483191285951007
auc: 0.5190422958640126
acc: 0.2702469250058393


In [21]:
cm = confusion_matrix(train_y.map(label_map), np.argmax(oof_lgb, axis=1))
acc0 = cm[0][0] / cm[0].sum()
acc1 = cm[1][1] / cm[1].sum()
acc2 = cm[2][2] / cm[2].sum()
acc3 = cm[3][3] / cm[3].sum()
cm
acc0, acc1, acc2, acc3

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
imp['score'] = imp['score1'] #+ imp['score2'] + imp['score3'] + imp['score4'] + imp['score5']
imp.sort_values(by='score', ascending=False)

In [ ]:
# imp_array = np.array(imp.sort_values(by='score', ascending=False)['feat'])
# imp_array
# np.save('E:/jet_complex_data/imp.npy', imp_array)

In [ ]:
test_jet = pd.read_feather(path+'jet_complex_data/complex_test_R04_jet.feather')

In [ ]:
test['label'] = preds.argmax(axis=1)
test['label'] = test['label'].map(label_map_reverse).astype(np.int64)

In [ ]:
sub = test_jet.merge(test[['event_id', 'label']], on='event_id', how='left')[['jet_id', 'label']]
sub.rename(columns={'jet_id':'id'}, inplace = True)
sub

In [ ]:
sub['label'].value_counts()
# 21    194142
# 1     135193
# 5     112430
# 4      96184

In [ ]:
# if not os.path.exists(path+'jet_complex_data/sub'):
#     os.mkdir(path+'jet_complex_data/sub')
# sub.to_csv(path+'jet_complex_data/sub/sub1-25_.csv', index=False)